In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

def calculate_protein_intake(weight, activity_level):
    activity_protein_factors = {
        'sedentary': 0.8,
        'lightly active': 1.0,
        'moderately active': 1.2,
        'very active': 1.5,
        'extra active': 1.8
    }
    protein_factor = activity_protein_factors.get(activity_level.lower(), 0.8)
    protein_intake_grams = protein_factor * weight
    return protein_intake_grams

def calculate_bmr(weight, height, age, gender):
    if gender.lower() == 'male':
        return 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    elif gender.lower() == 'female':
        return 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    else:
        return None

def calculate_tdee(bmr, activity_level):
    activity_factors = {
        'sedentary': 1.2,
        'lightly active': 1.375,
        'moderately active': 1.55,
        'very active': 1.725,
        'extra active': 1.9
    }
    factor = activity_factors.get(activity_level, 1.2)
    return bmr * factor

def calculate_bmi(weight, height):
    height_m = height / 100
    return weight / (height_m ** 2)

def suggest_caloric_adjustment(tdee, bmi):
    if bmi < 18.5:
        return tdee + 500, 'gain weight'
    elif 18.5 <= bmi < 24.9:
        return tdee, 'maintain weight'
    elif 25 <= bmi < 29.9:
        return tdee - 500, 'lose weight'
    else:
        return tdee - 1000, 'lose weight'

def calculate_fat_intake(calories):
    fat_calories = calories * 0.25
    fat_grams = fat_calories / 9
    return fat_calories, fat_grams

def calculate_fiber_intake(age):
    return age + 5

def calculate_sugar_intake(calories):
    return (calories * 0.1) / 4

def evaluate_model(food_data, user_nutrition, user_swipes):
    train_data, test_data = train_test_split(food_data, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    nutritional_columns = ['Calories', 'Protein', 'Carbohydrates', 'Fat', 'Fiber', 'Sugar', 'Sodium']
    train_nutrition = scaler.fit_transform(train_data[nutritional_columns])
    test_nutrition = scaler.transform(test_data[nutritional_columns])
    user_nutrition_df = pd.DataFrame([user_nutrition])
    user_nutrition_scaled = scaler.transform(user_nutrition_df)

    train_cos_sim = cosine_similarity(train_nutrition, user_nutrition_scaled).flatten()
    test_cos_sim = cosine_similarity(test_nutrition, user_nutrition_scaled).flatten()

    train_data['Similarity'] = train_cos_sim
    test_data['Similarity'] = test_cos_sim

    adjusted_train_data =adjust_for_swipes(train_data, user_swipes)
    adjusted_test_data = adjust_for_swipes(test_data, user_swipes)
        # Get top 5 recommendations
    top_recommendations = adjusted_test_data.sort_values(by='Similarity', ascending=False).head(5)
    top_categories_predicted = top_recommendations['Food Category'].tolist()

    return top_recommendations



print("Welcome to the daily calories calculator!")

weight = float(input("Enter your weight in kilograms: "))
height = float(input("Enter your height in centimeters: "))
age = int(input("Enter your age in years: "))
gender = input("Enter your gender (male/female): ")

bmr = calculate_bmr(weight, height, age, gender)

if bmr is not None:
    print("\nSelect your activity level:")
    print("1. Sedentary (little or no exercise)")
    print("2. Lightly active (light exercise/sports 1-3 days/week)")
    print("3. Moderately active (moderate exercise/sports 3-5 days/week)")
    print("4. Very active (hard exercise/sports 6-7 days a week)")
    print("5. Extra active (very hard exercise/sports & physical job or 2x training)")

    activity_level_choice = input("Enter the number corresponding to your activity level: ")

    activity_levels = {
        '1': 'sedentary',
        '2': 'lightly active',
        '3': 'moderately active',
        '4': 'very active',
        '5': 'extra active'
    }

    if activity_level_choice in activity_levels:
        activity_level = activity_levels[activity_level_choice]

        tdee = calculate_tdee(bmr, activity_level)

        bmi = calculate_bmi(weight, height)
        print(f"\nYour Body Mass Index (BMI) is: {bmi:.2f}")

        adjusted_calories, goal = suggest_caloric_adjustment(tdee, bmi)

        if tdee is not None:
            print(f"\nYour Total Daily Energy Expenditure (TDEE) is: {tdee:.2f} calories.")
            print(f"Based on your BMI, you should aim for: {adjusted_calories:.2f} calories per day to {goal}.")

            protein_intake_grams = calculate_protein_intake(weight, activity_level)
            print(f"\nTo {goal}, your recommended protein intake is: {protein_intake_grams:.2f} grams per day.")

            fat_calories, fat_grams = calculate_fat_intake(adjusted_calories)
            print(f"\nYou should consume no more than {fat_calories:.2f} calories from fat per day, which is about {fat_grams:.2f} grams of fat.")
            print(f"Only 10% of your fat intake should be from saturated fats, which is about {fat_grams * 0.10:.2f} grams of saturated fat.")

            fiber_intake = calculate_fiber_intake(age)
            print(f"\nYou should aim for about {fiber_intake:.2f} grams of fiber per day based on the 'age + 5' rule.")

            sugar_intake = calculate_sugar_intake(adjusted_calories)
            print(f"\nYou should consume no more than {sugar_intake:.2f} grams of sugar per day.")

            print("\nYour sodium intake should be less than 2000 mg per day.")

            lunch_calories = adjusted_calories / 3
            lunch_protein = protein_intake_grams / 3
            lunch_carbohydrates = (lunch_calories * 0.4 / 4) / 3
            lunch_fat_grams = fat_grams / 3
            lunch_fiber = fiber_intake / 3
            lunch_sugar = sugar_intake / 3
            lunch_sodium = 2000 / 3

            print("\nFor lunch, you should aim for:")
            print(f"Calories: {lunch_calories:.2f} kcal")
            print(f"Protein: {lunch_protein:.2f} grams")
            print(f"Carbohydrates: {lunch_carbohydrates:.2f} grams")
            print(f"Fat: {lunch_fat_grams:.2f} grams")
            print(f"Fiber: {lunch_fiber:.2f} grams")
            print(f"Sugar: {lunch_sugar:.2f} grams")
            print(f"Sodium: less than {lunch_sodium:.2f} mg")

            user_nutrition = {
                'Calories': lunch_calories,
                'Protein': lunch_protein,
                'Carbohydrates': lunch_calories * 0.4 / 4,  # Assuming 40% of calories from carbs
                'Fat': lunch_fat_grams,
                'Fiber': lunch_fiber,
                'Sugar': lunch_sugar,
                'Sodium': lunch_sodium
            }

            user_allergies = {
                'Egg-Free': int(input("Do you have an egg allergy? (1 for yes, 0 for no): ")),
                'Gluten Free': int(input("Do you have a gluten allergy? (1 for yes, 0 for no): ")),
                'Dairy Free': int(input("Do you have a dairy allergy? (1 for yes, 0 for no): ")),
                'Fish Free': int(input("Do you have a fish allergy? (1 for yes, 0 for no): ")),
                'Shellfish Free': int(input("Do you have a shellfish allergy? (1 for yes, 0 for no): ")),
                'Peanut free': int(input("Do you have a peanut allergy? (1 for yes, 0 for no): ")),
                'Treenut Free': int(input("Do you have a treenut allergy? (1 for yes, 0 for no): ")),
                'Soy Free': int(input("Do you have a soy allergy? (1 for yes, 0 for no): ")),
                'Wheat Free': int(input("Do you have a wheat allergy? (1 for yes, 0 for no): "))
            }

            user_swipes = {
                'Vegetable Dishes': 1 if input("Do you like Vegetable Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Chicken Dishes': 1 if input("Do you like Chicken Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Seafood Dishes': 1 if input("Do you like Seafood Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Pork Dishes': 1 if input("Do you like Pork Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Beef Dishes': 1 if input("Do you like Beef Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Noodle Dishes': 1 if input("Do you like Noodle Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Mixed Dishes': 1 if input("Do you like Mixed Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Breakfast Dishes': 1 if input("Do you like Breakfast Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Appetizers': 1 if input("Do you like Appetizers? (yes/no): ").lower() == 'yes' else -1,
                'Sandwiches': 1 if input("Do you like Sandwiches? (yes/no): ").lower() == 'yes' else -1,
                'Pasta Dishes': 1 if input("Do you like Pasta Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Pizza Dishes': 1 if input("Do you like Pizza Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Side Dishes': 1 if input("Do you like Side Dishes? (yes/no): ").lower() == 'yes' else -1,
                'Salads': 1 if input("Do you like Salads? (yes/no): ").lower() == 'yes' else -1
            }

            food_data = pd.read_csv('MenuDataFinal.csv')

            def filter_allergies(food_data, user_allergies):
                for allergy, has_allergy in user_allergies.items():
                    if has_allergy == 1:
                        food_data = food_data[food_data[allergy] == 1]
                return food_data

            filtered_food_data = filter_allergies(food_data, user_allergies)

            def filter_by_confidence(food_data, threshold=0.8):
                return food_data[food_data['Similarity'] >= threshold]

            scaler = StandardScaler()
            nutritional_columns = ['Calories', 'Protein', 'Carbohydrates', 'Fat', 'Fiber', 'Sugar', 'Sodium']
            food_nutrition = filtered_food_data[nutritional_columns]
            food_nutrition_scaled = scaler.fit_transform(food_nutrition)

            user_nutrition_df = pd.DataFrame([user_nutrition])
            user_nutrition_scaled = scaler.transform(user_nutrition_df)

            cos_sim = cosine_similarity(food_nutrition_scaled, user_nutrition_scaled).flatten()



            filtered_food_data['Similarity'] = cos_sim

            def adjust_for_swipes(food_data, user_swipes):
                for category, swipe in user_swipes.items():
                    if swipe != 0:
                        food_data.loc[food_data['Food Category'] == category, 'Similarity'] += swipe * 0.15
                return food_data

            adjusted_food_data = adjust_for_swipes(filtered_food_data, user_swipes)
            filtered_adjusted_food_data = filter_by_confidence(adjusted_food_data, threshold=0.8)

            top_recommendations = filtered_adjusted_food_data.sort_values(by='Similarity', ascending=False).head(5)
            print(top_recommendations)

            top_recommendations.to_csv('top_recommendations.csv', index=False)

            evaluate_model(filtered_food_data, user_nutrition, user_swipes)

        else:
            print("Error calculating TDEE.")
    else:
        print("Invalid activity level")
else:
    print("Error calculating BMR.")


Welcome to the daily calories calculator!
Enter your weight in kilograms: 80
Enter your height in centimeters: 171
Enter your age in years: 22
Enter your gender (male/female): male

Select your activity level:
1. Sedentary (little or no exercise)
2. Lightly active (light exercise/sports 1-3 days/week)
3. Moderately active (moderate exercise/sports 3-5 days/week)
4. Very active (hard exercise/sports 6-7 days a week)
5. Extra active (very hard exercise/sports & physical job or 2x training)
Enter the number corresponding to your activity level: 4

Your Body Mass Index (BMI) is: 27.36

Your Total Daily Energy Expenditure (TDEE) is: 3201.35 calories.
Based on your BMI, you should aim for: 2701.35 calories per day to lose weight.

To lose weight, your recommended protein intake is: 120.00 grams per day.

You should consume no more than 675.34 calories from fat per day, which is about 75.04 grams of fat.
Only 10% of your fat intake should be from saturated fats, which is about 7.50 grams of s

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelBinarizer

def evaluate_model(food_data):
    # Split data into training and testing sets
    train_data, test_data = train_test_split(food_data, test_size=0.2, random_state=42)

    # Filter test data to only include positive swipes
    positive_recommendations = test_data[test_data['Swipe'] == 1]

    # Display the positive food recommendations
    if not positive_recommendations.empty:
        top_recommendations = positive_recommendations.head(3)  # Get up to the top three
        print("Positive Recommendations:")
        print(top_recommendations['Food Category'])
    else:
        print("No positive recommendations based on user preferences.")

    # Prepare data for metrics evaluation
    lb = LabelBinarizer()
    true_labels = lb.fit_transform(test_data['Swipe'])
    predicted_labels = lb.transform(test_data['Food Category'].isin(top_recommendations['Food Category']).astype(int))

    # Calculate precision, recall, and F1-score
    precision = precision_score(true_labels, predicted_labels, average='micro')
    recall = recall_score(true_labels, predicted_labels, average='micro')
    f1 = f1_score(true_labels, predicted_labels, average='micro')

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")

# Example mock data with binary representation for categories
data = {
    'Food Name': [
        'Chopseuy', 'Chicken Inasal', 'Fried Bangus', 'Pork Belly', 'Beef Tapa',
        'Noodle Soup', 'Mixed Veggie', 'Eggs Benedict', 'Spring Rolls',
        'Ham Sandwich', 'Spaghetti', 'Margherita Pizza', 'Garlic Bread', 'Garden Salad'
    ],
    'Food Category': [
        'Vegetable Dishes', 'Chicken Dishes', 'Seafood Dishes', 'Pork Dishes', 'Beef Dishes',
        'Noodle Dishes', 'Mixed Dishes', 'Breakfast Dishes', 'Appetizers',
        'Sandwiches', 'Pasta Dishes', 'Pizza Dishes', 'Side Dishes', 'Salads'
    ],
    'Swipe': [
        1, 1, 0, 0, 0,
        0, 0, 1, 0,
        1, 1, 1, 1, 1  # Corresponds to the user's input preferences
    ]
}

# Create DataFrame
food_data = pd.DataFrame(data)

evaluate_model(food_data)


Positive Recommendations:
9           Sandwiches
11        Pizza Dishes
0     Vegetable Dishes
Name: Food Category, dtype: object
Precision: 1.00
Recall: 1.00
F1-Score: 1.00
